Steps to install uv are below. Note that each line is run sequentially:

```zsh
uv venv --python=python3.11
source .venv/bin/activate  # (On macOS/Linux)
# .venv\Scripts\activate  # (On Windows)
uv pip install geospatial
uv pip install dagster
uv pip install pyncei
pip list
pip freeze > requirements.txt  # Optional
```